In [2]:
from Imports import *

In [112]:
print_to_excel = True
print_to_Z = True

In [24]:
# read xlsx files from folder
Output_path = 'z:\\Anna_Klimova\\OCP\\Archive\\'
dir_list = os.listdir(Output_path)
xlsx_list = [x for x in dir_list if '.xlsx' in x]

# create pivot files - dates
xlsx_list_split = [x.split('_') for x in xlsx_list]
xlsx_split_df = pd.DataFrame(xlsx_list_split).sort_values([0]).reset_index(drop=True)
xlsx_split_pivot = xlsx_split_df.pivot_table(index=[1,2,3], values=0, aggfunc=list).reset_index()
xlsx_split_pivot = concat_columns(xlsx_split_pivot, [1,2,3]).drop(columns=[1,2,3])
xlsx_split_pairs = xlsx_split_pivot[xlsx_split_pivot[0].str.len()>1]

# files witch doesn't have a pair
xlsx_split_no_pairs = xlsx_split_pivot[xlsx_split_pivot[0].str.len()<2] 
xlsx_split_no_pairs['file_pre_name'] = xlsx_split_no_pairs[0].str[0] + '_' + xlsx_split_no_pairs['concat_columns']
xlsx_split_no_pairs['file_name'] = xlsx_split_no_pairs['file_pre_name'].apply(lambda x:[y for y in xlsx_list if x in y][0])
no_pairs_list = xlsx_split_no_pairs['file_name']

# create table with files names in 2 columns view
xlsx_split_pairs['first_date'] = xlsx_split_pairs[0].str[0] + '_' + xlsx_split_pairs['concat_columns']
xlsx_split_pairs['last_date'] = xlsx_split_pairs[0].str[1] + '_' + xlsx_split_pairs['concat_columns']
xlsx_split_pairs = xlsx_split_pairs.drop(columns=[0,'concat_columns'])
cols_list = ['first_date','last_date']
for col in cols_list:
    xlsx_split_pairs[f'{col}_file'] = xlsx_split_pairs[col].apply(lambda x:[y for y in xlsx_list if x in y][0])
xlsx_split_pairs = xlsx_split_pairs.drop(columns=cols_list)
xlsx_split_pairs

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_17612\2552251114.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xlsx_split_no_pairs['file_pre_name'] = xlsx_split_no_pairs[0].str[0] + '_' + xlsx_split_no_pairs['concat_columns']
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_17612\2552251114.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xlsx_split_no_pairs['file_name'] = xlsx_split_no_pairs['file_pre_name'].apply(lambda x:[y for y in xlsx_list if x in y][0])
C:\Users\KlimovaAnnaA\AppData\L

,first_date_file,last_date_file
1,2024-07-16_EUROCHEM_EX_Deals.xlsx,2024-08-13_EUROCHEM_EX_Deals.xlsx
2,2024-07-16_EUROCHEM_SAP_ARAP_Jun.xlsx,2024-08-13_EUROCHEM_SAP_ARAP_Jul.xlsx
3,2024-07-16_EUROCHEM_SAP_Debt_Jun.xlsx,2024-08-13_EUROCHEM_SAP_Debt_Jul.xlsx
4,2024-07-16_Ech_quantum_Debt_Jun.xlsx,2024-08-13_Ech_quantum_Debt_Jul.xlsx
5,2024-07-16_Ech_quantum_Deriv_Jun.xlsx,2024-08-13_Ech_quantum_Deriv_Jul.xlsx
6,2024-07-17_Invest_nv_EUROCHEM.xlsx,2024-08-13_Invest_nv_EUROCHEM.xlsx
7,2024-07-17_Invest_nv_SUEK.xlsx,2024-08-13_Invest_nv_SUEK.xlsx
8,2024-07-16_SUEK_SAP_ARAP_Jun.xlsx,2024-08-13_SUEK_SAP_ARAP_Jul.xlsx
9,2024-07-16_SUEK_SAP_Debt_Jun.xlsx,2024-08-13_SUEK_SAP_Debt_Jul.xlsx
10,2024-07-16_SUEK_quantum_Debt_Jun.xlsx,2024-08-13_SUEK_quantum_Debt_Jul.xlsx


In [25]:
xlsx_split_pairs = xlsx_split_pairs[~xlsx_split_pairs.first_date_file.str.contains('ARAP')].reset_index(drop=True)
# xlsx_split_pairs = xlsx_split_pairs[~xlsx_split_pairs.first_date_file.str.contains('invest_nv')].reset_index(drop=True)
# assert 2==1

In [107]:
# create df for compaire values
for file in [0,1]:
    compaire_df = pd.DataFrame()
    for counter in range(len(xlsx_split_pairs)):

        file_name = xlsx_split_pairs.iloc[counter,file]
        if 'ARAP' in file_name:
            file_data = pd.read_excel(Output_path+file_name, sheet_name='ARAP')
        else:
            file_data = pd.read_excel(Output_path+file_name)
        file_data_info = pd.DataFrame({'name':file_name,'columns':file_data.columns, 'types':file_data.dtypes, 'col_count':file_data.shape[1], 'str_count':file_data.shape[0], 'null_count':file_data.isna().sum(), 'zero_count':file_data[(file_data==0).all(axis=1)].count(), 'sum_values':file_data.astype(int, errors='ignore').sum()})
        file_data_info_values = file_data_info[(file_data_info['types'].astype(str).str.contains('float'))|(file_data_info['types'].astype(str).str.contains('int'))].reset_index(drop=True)
        compaire_df = pd.concat([compaire_df,file_data_info_values],axis=0)

        if file == 0:
            compaire_df_first = compaire_df
        if file == 1:
            compaire_df_last = compaire_df

compaire_df_first,compaire_df_last = compaire_df_first.reset_index(drop=True).sort_values('name'),compaire_df_last.reset_index(drop=True).sort_values('name')
compaire_df_first['merge_on'] = compaire_df_first['name'].apply(lambda x:'_'.join(x.split('_')[1:4])) 
compaire_df_last['merge_on'] = compaire_df_last['name'].apply(lambda x:'_'.join(x.split('_')[1:4]))
compaire_df = compaire_df_first.merge(compaire_df_last, how='outer', left_on=['merge_on', 'columns'], right_on=['merge_on', 'columns'], suffixes=['_1','_2'])
# compaire_df = compaire_df.drop(columns='merge_on')

In [110]:
# compaire_df['name_1'] = compaire_df['name_1'].fillna(method='ffill')
# compaire_df['name_2'] = compaire_df['name_2'].fillna(method='ffill')
compaire_df.head()
# compaire_df

,name_1,columns,types_1,col_count_1,str_count_1,null_count_1,zero_count_1,sum_values_1,merge_on,name_2,types_2,col_count_2,str_count_2,null_count_2,zero_count_2,sum_values_2
0,2024-07-16_EUROCHEM_EX_Deals.xlsx,Days,int64,12.0,12.0,0.0,0.0,4460,EUROCHEM_EX_Deals.xlsx,2024-08-13_EUROCHEM_EX_Deals.xlsx,int64,12.0,12.0,0.0,0.0,4088
1,2024-07-16_EUROCHEM_EX_Deals.xlsx,Sum_money,float64,12.0,12.0,0.0,0.0,-63712276,EUROCHEM_EX_Deals.xlsx,2024-08-13_EUROCHEM_EX_Deals.xlsx,float64,12.0,12.0,0.0,0.0,-63712276
2,2024-07-16_EUROCHEM_EX_Deals.xlsx,actualQuantity,int64,12.0,12.0,0.0,0.0,795245,EUROCHEM_EX_Deals.xlsx,2024-08-13_EUROCHEM_EX_Deals.xlsx,int64,12.0,12.0,0.0,0.0,795245
3,2024-07-16_EUROCHEM_SAP_Debt_Jun.xlsx,Days,int64,16.0,157.0,0.0,0.0,279955,EUROCHEM_SAP_Debt,2024-08-13_EUROCHEM_SAP_Debt_Jul.xlsx,int64,16.0,161.0,0.0,0.0,283571
4,2024-07-16_EUROCHEM_SAP_Debt_Jun.xlsx,amoutn_outstanding,float64,16.0,157.0,0.0,0.0,864741550279.340088,EUROCHEM_SAP_Debt,2024-08-13_EUROCHEM_SAP_Debt_Jul.xlsx,float64,16.0,161.0,0.0,0.0,879118936082.329956


In [108]:
compaire_df_work = compaire_df[:].fillna(0)
# compaire values
columns_list_eq = ['types','col_count']
columns_list_mostly_eq = ['null_count','sum_values','str_count','zero_count']
for col in columns_list_eq:
    compaire_df_work[f'{col}_is_equal'] = compaire_df_work[f'{col}_1']==compaire_df_work[f'{col}_2']
for col in columns_list_mostly_eq:
    tolerance = 0.4
    max_value = compaire_df_work[compaire_df_work.columns[compaire_df_work.columns.str.contains(col)]].abs().max(axis=1)
    compaire_df_work[f'{col}_is_equal'] = (abs(compaire_df_work[f'{col}_1']-compaire_df_work[f'{col}_2'])/max_value)<=tolerance
    compaire_df_work.loc[compaire_df_work[f'{col}_1']==compaire_df_work[f'{col}_2'], f'{col}_is_equal'] = True

# choose where False
is_equlal_columns = compaire_df_work.columns[compaire_df_work.columns.str.contains('is_equal')].tolist()
compaire_df_work_false = compaire_df_work[(compaire_df_work[is_equlal_columns]==False).any(axis=1)]
compaire_df_work_false_pivot = compaire_df_work_false.pivot_table(index='merge_on', values=is_equlal_columns, aggfunc='min').reset_index()

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_17612\581079492.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  compaire_df_work = compaire_df[:].fillna(0)


In [120]:
if print_to_excel == True:
    Output_file = f'{str(date.today())}_compaire_files.xlsx'
    if print_to_Z == True:
        Output_file = Output_path + Output_file
    pd.DataFrame(no_pairs_list).to_excel(Output_file, sheet_name='No_pairs', index=False)
    new_list(compaire_df_work_false_pivot, Output_file, 'Files_with_problems')
    new_list(compaire_df_work, Output_file, 'Compaire_files')